In [ ]:
%pip install ingredient_parser_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 21.1 MB/s eta 0:00:00


In [ ]:
from ingredient_parser import parse_multiple_ingredients

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
import pandas as pd
file_path = "sample_data/final_recipes.xlsx"
# Read the CSV file and handle the error by skipping the problematic row
# Read the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(file_path)
    # Now 'df' contains the data from the Excel file
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")

In [ ]:
df

,Unnamed: 0.1,Unnamed: 0,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing
0,0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","['whole chicken', 'kosher salt', 'acorn squash..."
1,1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (...","['egg whites', 'new potatoes', 'kosher salt', ..."
2,2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,"['1 cup evaporated milk', '1 cup whole milk', ...","['evaporated milk', 'whole milk', 'garlic powd..."
3,3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,"['1 (¾- to 1-pound) round Italian loaf, cut in...","['round italian', 'olive oil', 'sweet italian ..."
4,4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","['dark brown sugar', 'hot water', 'bourbon', '..."
...,...,...,...,...,...,...,...
13496,13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,"['1 cup all-purpose flour', '2/3 cup unsweeten...","['all-purpose flour', 'unsweetened cocoa powde..."
13497,13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,"['1 preserved lemon', '1 1/2 pound butternut s...","['preserved lemon', 'butternut squash', 'olive..."
13498,13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",['Leftover katsuo bushi (dried bonito flakes) ...,"['sake', 'sugar', 'soy sauce', 'sesame seeds',..."
13499,13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,"['unsalted butter', 'baby spinach', 'feta', 'n..."


In [ ]:
columns_to_remove= ["Unnamed: 0"]
df = df.drop(columns=columns_to_remove, axis=1)

In [ ]:
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
!pip install rank_bm25
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
def preprocess(text):
    #define stopwords
    stop_words = set(stopwords.words("english"))
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
    #Convert to lowercase
    text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    ##Convert to list from string
    text = text.split()
    ##Stemming
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  stop_words]
    text = " ".join(text)

    return text

In [ ]:
df['try_processs'] = df['final_ing'].apply(lambda x:(preprocess(x)))

In [ ]:
pt = list(df['try_processs'])

In [ ]:
pt[:10]

['whole chicken kosher salt acorn squash sage rosemary unsalted butter ground allspice crushed red pepper flake freshly ground black pepper white bread apple extra virgin olive oil red onion apple cider vinegar white miso purpose flour unsalted butter dry white wine unsalted chicken broth white miso kosher salt freshly ground pepper',
 'egg white new potato kosher salt finely ground black pepper rosemary thyme parsley',
 'evaporated milk whole milk garlic powder onion powder smoked paprika freshly ground black pepper kosher salt extra sharp cheddar full fat cream cheese elbow macaroni',
 'round italian olive oil sweet italian sausage unsalted butter onion celery garlic egg heavy cream turkey giblet stock reduced sodium chicken broth parmigiano reggiano flat leaf parsley qt shallow ceramic glass baking dish',
 'dark brown sugar hot water bourbon fresh lemon juice apple butter garnish orange twist ground cinnamon',
 'chamomile tea bag reposado tequila fresh lemon juice agave nectar',
 'g

In [ ]:
%pip install rank_bm25
from rank_bm25 import BM25Okapi

In [ ]:
from rank_bm25 import BM25Okapi
tokenized_corpus = [doc.split(" ") for doc in pt]
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
query = "egg whites, new potatoes, kosher salt, finely ground black pepper, rosemary, thyme, parsley"
tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)
bm25.get_top_n(tokenized_query, pt, n=10)

['egg white new potato kosher salt finely ground black pepper rosemary thyme parsley',
 'lamb shank onion garlic paprika ground cumin ground ginger finely ground black pepper saffron thread sea salt extra virgin olive oil flat leaf parsley cilantro new potato fresh petits poi moroccan bread',
 'fine plain dried breadcrumb kosher salt baking powder finely ground black pepper russet potato onion egg schmaltz duck fat vegetable oil',
 'yukon gold potato egg yolk egg sour cream purpose flour kosher salt finely ground black pepper baking soda ground nutmeg unsalted butter food mill potato ricer',
 'unsalted butter kosher salt baking powder ground cardamom ground clove finely ground black pepper purpose flour granulated sugar lemon zest vanilla bean egg egg yolk powdered sugar red currant jelly',
 'red white new potato parsnip milk butter parsley cayenne pepper several grinding black pepper salt',
 'sweet potato large yam kosher salt banana unsalted butter heavy cream cinnamon light brown su

In [ ]:

tokenized_query = query.split(" ")
doc_scores = bm25.get_scores(tokenized_query)

query = "egg whites, new potatoes, kosher salt, finely ground black pepper, rosemary, thyme, parsley"
print(tokenized_query)
# Define the target value as a list
target_values = bm25.get_top_n(tokenized_query, pt, n=5)

# Use boolean indexing to filter rows
result_df = pd.DataFrame(columns=df.columns)

for tv in target_values:
    # Use boolean indexing to filter rows with exactly the target value
    filtered_rows = df[df['try_processs'] == tv]
    result_df = result_df._append(filtered_rows,ignore_index=True)



['egg', 'whites,', 'new', 'potatoes,', 'kosher', 'salt,', 'finely', 'ground', 'black', 'pepper,', 'rosemary,', 'thyme,', 'parsley']


In [ ]:
df

,Unnamed: 0.1,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing,try_processs
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","['whole chicken', 'kosher salt', 'acorn squash...",whole chicken kosher salt acorn squash sage ro...
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (...","['egg whites', 'new potatoes', 'kosher salt', ...",egg white new potato kosher salt finely ground...
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,"['1 cup evaporated milk', '1 cup whole milk', ...","['evaporated milk', 'whole milk', 'garlic powd...",evaporated milk whole milk garlic powder onion...
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,"['1 (¾- to 1-pound) round Italian loaf, cut in...","['round italian', 'olive oil', 'sweet italian ...",round italian olive oil sweet italian sausage ...
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,"['1 teaspoon dark brown sugar', '1 teaspoon ho...","['dark brown sugar', 'hot water', 'bourbon', '...",dark brown sugar hot water bourbon fresh lemon...
...,...,...,...,...,...,...,...
13496,13496,Brownie Pudding Cake,"['1 cup all-purpose flour', '2/3 cup unsweeten...",Preheat the oven to 350°F. Into a bowl sift to...,"['1 cup all-purpose flour', '2/3 cup unsweeten...","['all-purpose flour', 'unsweetened cocoa powde...",purpose flour unsweetened cocoa powder double ...
13497,13497,Israeli Couscous with Roasted Butternut Squash...,"['1 preserved lemon', '1 1/2 pound butternut s...",Preheat oven to 475°F.\nHalve lemons and scoop...,"['1 preserved lemon', '1 1/2 pound butternut s...","['preserved lemon', 'butternut squash', 'olive...",preserved lemon butternut squash olive oil oni...
13498,13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,['Leftover katsuo bushi (dried bonito flakes) ...,"If using katsuo bushi flakes from package, moi...",['Leftover katsuo bushi (dried bonito flakes) ...,"['sake', 'sugar', 'soy sauce', 'sesame seeds',...",sake sugar soy sauce sesame seed japanese shor...
13499,13499,Spanakopita,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,Melt 1 tablespoon butter in a 12-inch heavy sk...,['1 stick (1/2 cup) plus 1 tablespoon unsalted...,"['unsalted butter', 'baby spinach', 'feta', 'n...",unsalted butter baby spinach feta nutmeg phyll...


In [ ]:
result_df

,Unnamed: 0.1,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing,try_processs
0,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (...","['egg whites', 'new potatoes', 'kosher salt', ...",egg white new potato kosher salt finely ground...
1,1244,Lamb Tagine with Potatoes and Peas (L'Ham Bel ...,['4 lamb shanks (3 pounds 5 ounces/1.5 kg tota...,"Put the shanks, onions, garlic, spices, and a ...",['4 lamb shanks (3 pounds 5 ounces/1.5 kg tota...,"['lamb shanks', 'onions', 'garlic', 'paprika',...",lamb shank onion garlic paprika ground cumin g...
2,3282,Silver Dollar Latkes,"['1/4 cup fine plain dried breadcrumbs', '1 ta...",Place a wire rack in a rimmed baking sheet; li...,"['1/4 cup fine plain dried breadcrumbs', '1 ta...","['fine plain dried breadcrumbs', 'kosher salt'...",fine plain dried breadcrumb kosher salt baking...
3,2343,Potato Blinis,"['1 pound Yukon Gold potatoes, scrubbed', '3 l...",Preheat oven to 425°F. Prick potatoes all over...,"['1 pound Yukon Gold potatoes, scrubbed', '3 l...","['yukon gold potatoes', 'egg yolks', 'egg', 's...",yukon gold potato egg yolk egg sour cream purp...
4,3290,Spiced Brown Butter Linzer Cookies,"['1 1/2 cups (3 sticks) unsalted butter', '3/4...",Cook butter in a large saucepan over medium he...,"['1 1/2 cups (3 sticks) unsalted butter', '3/4...","['unsalted butter', 'kosher salt', 'baking pow...",unsalted butter kosher salt baking powder grou...


##LLM

In [ ]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece
!pip install evaluate
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
!nvidia-smi

Sun Dec  3 15:39:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Load Model

In [ ]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer

# the model name and loading setting
load_in_8bit = True
model_name = "lmsys/vicuna-7b-v1.5" # You can also use different LLMs like "mistralai/Mistral-7B-v0.1" for better performance

# Q2: Load the model and the tokenizer (2 points)


model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=load_in_8bit)
tokenizer = AutoTokenizer.from_pretrained(model_name,load_in_8bit=load_in_8bit)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
list=[['whole chicken', 'kosher salt', 'acorn squash', 'sage', 'rosemary', 'unsalted butter', 'ground allspice', 'crushed red pepper flakes', 'freshly ground black pepper', 'white bread', 'apples', 'extra-virgin olive oil', 'red onion', 'apple cider vinegar', 'white miso', 'all-purpose flour', 'unsalted butter', 'dry white wine', 'unsalted chicken broth', 'white miso', 'kosher salt', 'freshly ground pepper'],
['egg whites', 'new potatoes', 'kosher salt', 'finely ground black pepper', 'rosemary', 'thyme', 'parsley'],
['evaporated milk', 'whole milk', 'garlic powder', 'onion powder', 'smoked paprika', 'freshly ground black pepper', 'kosher salt', 'extra-sharp cheddar', 'full-fat cream cheese', 'elbow macaroni'],
['round italian', 'olive oil', 'sweet italian sausage', 'unsalted butter', 'onions', 'celery', 'garlic', 'eggs', 'heavy cream', 'turkey giblet stock or reduced-sodium chicken broth', 'parmigiano-reggiano', 'flat-leaf parsley', 'qt shallow ceramic or glass baking dish'],
['dark brown sugar', 'hot water', 'bourbon', 'fresh lemon juice', 'apple butter', 'garnish : orange twist and, or ground cinnamon'],
['chamomile tea bags', 'reposado tequila', 'fresh lemon juice', 'agave nectar'],
['grand marnier', 'amaro averna', 'pat salted butter', 'hot apple cider', 'fresh lemon juice', 'garnish : freshly ground pink peppercorns'],
['granulated sugar', 'ground turmeric', 'amontillado sherry', 'bourbon, scotch', 'turmeric syrup', 'fresh lemon juice', 'garnish : dehydrated lemon wheel'],
['assorted dals', 'white jasmine rice or other long-grain rice', 'pearl barley', 'bone-in lamb stew meat', 'kosher salt', 'fresh ginger', 'shallots', 'ghee or vegetable oil', 'garlic', 'garam masala', 'cayenne pepper', 'ground turmeric', 'green thai chiles', 'cilantro', 'white onion', 'limes'],
['basic lentil soup', 'onion', 'turmeric', 'cumin', 'aleppo pepper or, red pepper flakes', 'tomato paste', 'eggs', 'ghee, unsalted butter', 'whole cumin seeds', 'olive oil', 'kosher salt and freshly ground black pepper', 'parsley'],
['chipotle in adobo sauce', 'garlic', 'kosher salt', 'cornstarch', 'evaporated milk', 'sharp or extra-sharp cheddar cheese', 'cream cheese', 'mayonnaise', 'pimento peppers', 'chipotle pepper purée'],
['bone-in', 'parsnips', 'carrots', 'maple syrup', 'unsweetened apple juice', 'dark brown sugar', 'smooth dijon mustard', 'grainy dijon mustard', 'apple cider vinegar', 'kosher salt'],
['bacon', 'celery', 'carrot', 'onion', 'cinnamon', 'boiling potatoes', 'granny smith apples', 'butternut squash', 'reduced-sodium chicken stock or broth', 'water', 'sour cream'],
['anchovies', 'garlic', 'mayonnaise', 'dijon mustard', 'extra-virgin olive oil', 'freshly ground black pepper', 'whole chicken', 'kosher salt', 'shallots', 'lemons', 'parmesan', 'country-style bread', 'romaine hearts'],
['skinless, boneless chicken thighs', 'kosher salt, freshly ground pepper', 'unsalted butter', 'leeks', 'zest and, lemon', 'long-grain white rice', 'low-sodium chicken broth', 'oil-packed anchovy fillet', 'garlic', 'capers', 'red pepper flakes', 'tender herb leaves', 'extra-virgin olive oil'],
['instant corn masa flour', 'lard, vegetable oil', 'diamond crystal or, morton kosher salt', 'guajillo or dried new mexico chiles', 'morita chiles', 'ground coriander or cumin', 'medium shrimp', 'garlic', 'thyme', 'apple cider vinegar', 'agave syrup or honey', 'unsalted butter', 'onion', 'avocado', 'cilantro leaves', 'and lime wedges'],
['vegetable oil', 'corn tortillas', 'fresh chorizo', 'garlic', 'white onion', 'kosher salt', 'black beans', 'low-sodium chicken broth', 'queso fresco or cotija cheese', 'cilantro leaves, avocado'],
['virgin coconut oil', 'ripe, plátanos manzanos or, ripe (black) plantains', 'kosher salt', 'plain unsweetened coconut milk yogurt or greek yogurt', 'ripe mangoes', 'toasted nuts', 'seeds', 'and / or unsweetened, coconut'],
['skin-on', 'kosher salt', 'potatoes', 'shallots', 'unsalted butter', 'heavy cream', 'garlic', 'freshly ground black pepper', 'or ground nutmeg', 'thyme', 'sage leaves', 'country-style bread', 'lemon'],
['of zest no pith, orange', 'fresh ginger', 'fresh lime juice', 'champagne vinegar', 'sugar', 'kosher salt', 'extra-virgin olive oil', 'thai chile', 'beets', 'extra-virgin olive oil', 'white wine vinegar', 'water', 'kosher salt and freshly ground black pepper', 'extra-virgin olive oil', 'cashews', 'sunchokes', 'vegetable oil', 'kosher salt', 'orange', 'roasted beets', 'extra-virgin olive oil', 'kosher salt', 'pickled orange-ginger puree', 'cashews', 'tarragon', 'dill fronds', 'sunchoke chips'],
['toasted pepitas', 'kale', 'garlic', 'chickpeas', 'ground turmeric', 'coriander (cilantro) leaves', 'sea salt and cracked black pepper', 'pumpkin', 'extra virgin olive oil', 'plain greek-style yogurt and hulled tahini', 'carrots', 'apple cider vinegar', 'honey', 'red chiles', 'coriander (cilantro) leaves'],
['butternut squash', 'extra virgin olive oil', 'pure maple syrup', 'chile flakes', 'sea salt and cracked black pepper', 'almonds', 'quinoa', 'mint leaves', 'wild rocket (arugula) leaves', 'hulled tahini', 'lemon juice', 'water', 'garlic', 'sea salt flakes'],
['ghee', 'paneer', 'confectioners’ sugar', 'ground cardamom', 'semolina', 'ghee', 'whole milk', 'cashews', 'golden raisins', 'granulated sugar', 'freshly whipped cream, almonds'],
['whole milk', 'lemons', 'kosher salt'],
['carrot', 'tomatoes', 'extra virgin olive oil', 'garlic', 'fresh spinach', 'kosher salt', 'ground turkey', 'ground cumin', 'smoked paprika', 'ground black pepper', 'r-rated onions', 'red wine vinegar', 'eggs parmigiano reggiano'],
['yellow or white onions', 'olive or vegetable oil', 'kosher salt'],
['extra-virgin olive oil', 'white miso', 'pure maple syrup or brown sugar', 'regular soy sauce or tamari', 'kabocha squash', 'extra-virgin olive oil', 'ginger', 'garlic', 'low-sodium vegetable broth', 'white miso', 'kosher salt, freshly ground pepper', 'broccolini', 'wavy ramen noodles', 'cilantro leaves with tender stems'],
['coconut oil', 'calabaza pumpkin', 'onion', 'garlic', 'ají dulce pepper', 'dark brown sugar', 'ground cumin', 'coconut milk', 'chicken broth or vegetable broth', 'salt and freshly ground black pepper', 'fresh cilantro'],
['italian eggplants', 'canola oil', 'onion', 'garlic', 'tomato', 'coarse salt', 'freshly ground black pepper', 'cilantro', 'roti'],
['chicken', 'lime juice', 'green seasoning', 'coarse salt', 'scotch bonnet pepper or other hot red chili pepper', 'garlic', 'trinidad curry powder', 'canola oil', 'onion', 'chicken stock or water', 'roti or white rice'],
['cardamom pods', 'coriander seeds', 'cumin seeds', 'mustard seeds', 'fenugreek seeds', 'curry leaves', 'whole black peppercorns', 'whole cloves', 'ground turmeric'],
['garlic', 'onions', 'celery', 'red bell pepper', 'green bell pepper', 'olive oil', 'chicken broth', 'whole tomatoes', 'bay leaf', 'ground cumin', 'salt', 'freshly ground black pepper', 'large shrimp', 'accompaniment, uncooked rice', 'prepared according'],
['fresh chives', 'fresh shado beni or cilantro leaves', 'fresh thyme', 'fresh oregano', 'fresh parsley', 'garlic'],
['dry or 1, pigeon peas, pinto beans', 'long-grain rice', 'canola oil', 'sugar', 'chicken', 'onion', 'garlic', 'coconut milk', 'bay leaf', 'green seasoning', 'parsley', 'thyme', 'carrots', 'scallions', 'kosher salt', 'fresh calabaza or butternut squash', 'whole scotch bonnet pepper', 'ketchup', 'butter'],
['yukon gold potatoes', 'garlic', 'kosher salt', 'whole milk', 'thyme', 'unsalted butter', 'freshly ground black pepper', 'sour cream', 'a potato ricer or food mill'],
['london dry gin', 'cynar', 'fino sherry', 'orange twist'],
['graham cracker crumbs', 'granulated sugar', 'light brown sugar', 'kosher salt', 'unsalted butter', 'cream cheese', 'granulated sugar', 'creole cream cheese', 'vanilla bean paste', 'eggs', 'ingredient info, sour cream mixed with, buttermilk, creole cream cheese.', 'unsalted butter', 'granulated sugar', 'light brown sugar', 'honey', 'granny smith apples', 'lemon zest', 'fresh lemon juice', 'kosher salt', 'ground cinnamon', 'ground cardamom', 'ground ginger', 'vanilla extract'],
['skim milk', 'buttermilk', 'rennet tablet'],
['nonfat dry milk powder', 'baking powder', 'granulated sugar or light brown sugar', 'all-purpose flour', 'vegetable oil', 'vegetable shortening', 'ghee', 'unsalted butter', 'butter'],
['eggs', 'mayonnaise', 'dijon mustard', 'cayenne', 'garnishes : paprika and, fresh chives', 'pastry, star tip'],
['green, unripe mangoes', 'shallots', 'indian or thai green chiles', 'ginger', 'garlic', 'coconut vinegar or white vinegar', 'salt', 'coconut milk', 'coconut oil', 'black mustard seeds', 'dried red chiles', 'curry leaves', 'fried onions', 'basmati rice'],
['piloncillo or, dark brown sugar', 'cardamom pods', 'allspice berries', 'black peppercorns', 'kosher salt', 'orange zest', 'cold-brew coffee', 'unsweetened coconut milk or half-and-half'],
['eggs', 'pumpkin purée', 'milk', 'unsalted butter', 'all-purpose flour', 'cinnamon', 'allspice', 'ground ginger', 'nutmeg', 'kosher salt', 'apple cider or juice', 'maple syrup', 'powdered sugar'],
['chicken legs', 'kosher salt', 'extra-virgin olive oil', 'fennel', 'red flame or green table grapes', 'unsalted chicken broth', 'honey', 'calabrian chile paste or, calabrian chiles', 'red wine vinegar'],
['onion', 'unsalted butter', 'all-purpose flour', 'milk', 'boiling potatoes', 'sharp cheddar', 'dry bread crumbs'],
['skinless salmon fillet', 'greek yogurt', 'garlic granules', 'rose harissa', 'ground turmeric', 'paprika', 'zest of, unwaxed lime and', 'olive oil', 'maldon sea salt flakes and freshly ground black pepper', 'tortilla wraps', 'sliced tomatoes', 'onion', 'coriander leaves', 'greek yogurt'],
['water', 'instant yeast', 'bread flour', 'canola oil', 'morton kosher salt'],
['beets', 'white vinegar', 'boneless pork butt', 'beef, chicken', 'bay leaf', 'whole allspice berries', 'whole black peppercorns', 'carrots', 'celery', 'green cabbage', 'idaho potatoes', 'lima beans', 'salt'],
['skin-on red snapper fillets', 'okra salt or kosher salt', 'alligator pepper or freshly ground black pepper', 'cubeb pepper or ashanti pepper', 'green or medium-ripe papaya', 'cilantro', 'extra-virgin olive oil', 'red onion', 'garlic', 'ginger', 'ground dried bird chiles or other hot chile powder', 'lemon', 'puna, white yams, sweet potatoes', 'coconut sugar or dark brown sugar', 'kosher salt', 'extra-virgin olive oil', 'freshly ground black pepper', 'chile powder', 'drunk apricot shito', 'a spice mill or mortar and pestle'],
['dried apricots', 'dark rum or arrack', 'unsalted butter', 'ground cinnamon', 'coconut sugar or dark brown sugar', 'grains of selim pods', 'cubeb pepper or ashanti pepper', 'peanut oil or vegetable oil', 'white onion', 'red onion', 'garlic', 'ginger', 'red pepper flakes', 'rosemary', 'thyme', 'dried prawn or shrimp powder', 'okra salt or kosher salt', 'tomato purée', 'dried chile powder', 'low-sodium chicken broth', 'scotch bonnet or habanero chile', 'a spice mill or mortar and pestle'],
['dried beans', 'dried kombu', 'tomatoes', 'celery', 'carrot', 'leek', 'shallots', 'garlic', 'oil-packed anchovy fillets', 'extra-virgin olive oil', 'maple syrup', 'kosher salt', 'red pepper flakes', 'whole-grain mustard', 'bacon', 'romano beans or green beans', 'shallots', 'red pepper flakes', 'zest and, lemon', 'garlic', 'pickled green beans', 'cherry tomatoes', 'extra-virgin olive oil', 'sherry vinegar', 'tomato brown butter', 'feta', 'parsley leaves', 'and marjoram leaves'],
['tomato', 'unsalted butter', 'kosher salt', 'freshly ground black pepper'],
['acorn squash', 'unsalted butter', 'pure maple syrup', 'good olive oil', 'kosher salt and freshly ground black pepper', 'flaked sea salt'],
['extra-virgin olive oil', 'serrano chile or jalapeño', 'fresh ginger', 'garlic', 'scallions', 'ground turmeric', 'corn', 'grains', 'unsweetened coconut milk', 'kosher salt', 'store-bought crispy onions or shallots', 'lime wedges'],
['mixed heirloom tomatoes', 'extra-virgin olive oil', 'kosher salt', 'freshly ground black pepper', 'garlic', 'unsalted butter', 'stoned wheat thins or other whole grain crackers', 'eggs', 'taleggio cheese', 'parmesan', 'mayonnaise', 'thyme', 'shallot', 'special equipment, springform pan'],
['scallions', 'ginger', 'garlic', 'coriander seeds', 'cumin seeds', 'black mustard seeds', 'kashmiri chile powder or, cayenne powder', 'ground turmeric', 'extra-virgin olive oil', 'paneer, planks', 'kosher salt', 'cherry tomatoes', 'sugar snap peas', 'sugar', 'mint leaves', 'rice'],
['skinless, boneless chicken breasts', 'kosher salt', 'scallions', 'persian cucumbers', 'sesame seeds', 'fresh lime juice', 'tahini', 'soy sauce', 'honey', 'red pepper flakes', 'romaine or green or red leaf lettuce, little gem lettuce', 'cilantro leaves with tender stems', 'lime'],
['extra-virgin olive oil', 'garlic', 'double-concentrated tomato paste', 'pure maple syrup', 'soy sauce', 'worcestershire sauce', 'unseasoned rice vinegar', 'sriracha or other hot sauce', 'vegetable oil', 'whole chicken', 'kosher salt'],
['egg yolks', 'kefir (cultured milk) or plain whole-milk yogurt', 'dill fronds with tender stems', 'garlic', 'unsalted butter', 'pine nuts, almonds', 'dried currants or raisins', 'kosher salt', 'ground cumin', 'freshly ground black pepper', 'ground lamb', 'orecchiette or other short pasta', 'lemon'],
['short-grain sushi rice', 'scallions', 'ginger', 'vegetable oil', 'skinless, boneless cod', 'kosher salt', 'freshly ground black pepper', 'unseasoned rice vinegar', 'sugar', 'unsalted butter'],
['boneless, skinless center-cut salmon', 'scallions', 'ginger', 'garlic', 'mayonnaise', 'kosher salt', 'sesame oil', 'unseasoned rice vinegar', 'persian cucumbers', 'serrano chile', 'sugar', 'vegetable oil', 'rice flour', 'tender herbs', 'watercress', 'sesame seeds', 'brioche buns'],
['guajillo chiles', 'morita chiles', 'skinless striped bass or halibut fillets', 'fresh orange juice', 'achiote paste', 'garlic, whole', 'fresh lime juice', 'kosher salt', 'red onion', 'habanero chile', 'pineapple', 'cilantro', 'vegetable oil', 'corn tortillas', 'lime wedges'],
['raw nuts', 'raw pumpkin seeds', 'extra-virgin olive oil', 'old-fashioned oats', 'dried fruit', 'unsalted or low-salt natural peanut butter', 'honey', 'kosher salt', 'vanilla extract', 'egg white'],
['whole-milk yogurt', 'fresh lemon juice', 'sugar', 'kosher salt', 'red lentils', 'zucchini', 'onion', 'kosher salt', 'kashmiri chile powder or, cayenne pepper', 'ground turmeric', 'parsley leaves', 'lemon zest', 'ghee or neutral vegetable oil'],
['vegetable oil', 'cooking onions'],
['baby yukon gold potatoes', 'kosher salt', 'sour cream', 'mayonnaise', 'onion powder', 'dijon mustard', 'freshly ground black pepper', 'garlic', 'chives', 'red onion', 'sour cream and onion potato chips'],
['ground coriander', 'ground cumin', 'ground turmeric', 'red pepper flakes', 'extra-virgin olive oil', 'garlic', 'skinless, boneless chicken thighs', 'kosher salt', 'fresh lime juice', 'ripe avocados', 'ripe mangoes', 'red onion', 'torn little gem or green lettuce leaves', 'cherry tomatoes', 'flaky sea salt', 'mild red pepper flakes', 'and basil and / or mint leaves'],
['golden raisins', 'thin-skinned cucumbers', 'kosher salt', 'extra-virgin olive oil', 'pine nuts', 'castelvetrano olives', 'egg', 'garlic', 'panko', 'parsley', 'red pepper flakes', 'freshly ground black pepper', 'ground pork', 'red onion', 'tender herb leaves', 'red wine vinegar', 'plain whole-milk greek yogurt'],
['butter', 'whole graham crackers', 'digestive biscuits', 'sweetened condensed milk', 'hass avocados', 'cream cheese', 'fine sea salt', 'lemons', 'limes'],
['all purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'sugar', 'unsalted butter', 'vegetable oil', 'eggs', 'vanilla extract', 'buttermilk', 'unpeeled zucchini', 'semisweet chocolate chips', 'walnuts'],
['hazelnuts', 'bow tie pasta', 'unsalted butter', 'garlic', 'salt and freshly ground black pepper', 'crushed red pepper flakes', 'swiss chard', 'parmesan cheese', 'balsamic vinegar'],
['olive oil', 'garlic', 'cherry tomatoes', 'dry white wine', 'fresh basil leaves', 'fine sea salt', 'freshly ground black pepper', 'cooking fat', 'scallions', 'garlic', 'potato', 'fresh, frozen corn kernels', 'eggs', 'red pepper flakes', 'feta cheese'],
['all-purpose flour', 'salt', 'dried parsley', 'unsalted butter', 'sour cream or plain full-fat yogurt', 'white wine vinegar', 'ice water', 'cooking fat', 'onion', 'mixed heirloom tomatoes', 'hard salty cheese', 'mozzarella cheese', 'mayonnaise', 'basil leaves', 'parsley', 'thyme', 'salt and freshly ground black pepper', 'egg', 'milk', 'cream', 'or water'],
['" bone-in pork chops', 'kosher salt', 'coriander seeds', 'small padrón chiles or shishito peppers', 'red onion', 'garlic', 'carrots', 'red wine vinegar', 'sherry vinegar or red wine vinegar', 'honey', 'paprika', 'extra-virgin olive oil', 'oregano'],
['dashi', 'doenjang', 'gochujang', 'gochugaru', 'garlic', 'soy sauce', 'onions', 'green zucchini', 'red korean chili pepper or other long hot chili', 'green korean chili pepper or other long hot chili', 'brisket', 'soft or silken tofu', 'scallions'],
['chicken wingettes, chicken', 'kosher salt', 'ground black pepper', 'potato starch', 'vegetable oil', 'peanuts', 'rice syrup or honey', 'soy sauce', 'brown or white sugar', 'white vinegar', 'yellow mustard', 'vegetable oil', 'garlic', 'peeled ginger', 'dried red chili peppers', 'sesame seeds', 'crushed red pepper flakes'],
['onion', 'gochujang', 'soy sauce', 'garlic', 'sugar', 'sake', 'mirin', 'sesame oil', 'skinless pork belly', 'unsalted butter', 'slider buns', 'mayonnaise', 'english cucumbers'],
['whole chicken', 'white or black peppercorns', 'garlic', 'cilantro roots', 'light brown sugar', 'fish sauce', 'ground turmeric', 'kosher salt', 'ground coriander', 'ground cumin', 'ground cardamom', 'fresh ripe pineapple', 'distilled white vinegar or apple cider vinegar', 'garlic', 'red jalapeños or serrano chiles', 'granulated sugar', 'kosher salt', 'freshly ground white pepper', 'saffron threads, ground turmeric', 'granulated sugar', 'kosher salt', 'virgin coconut oil', 'vegetable oil', 'cilantro', 'jasmine rice, coconut rice'],
['egg yolks', 'all-purpose flour', 'almond flour, all-purpose flour', 'sugar', 'kosher salt', 'unsalted butter', 'whole milk', 'vanilla paste or, vanilla extract', 'egg yolks', 'sugar', 'cornstarch', 'kosher salt', 'unsalted butter', 'fresh blueberries', 'fresh blackberries', 'special equipment, springform pan or tart pan'],
['raw thai jasmine rice', 'coconut milk', 'water', 'granulated sugar', 'salt', 'coconut cream'],
['raw sesame seeds', 'lemon', 'peaches ', 'sugar', 'kosher salt', 'all-purpose flour', 'unsalted butter', 'ground cinnamon', 'tahini', 'sesame oil', 'cornstarch', 'vanilla ice cream'],
['hazelnuts', 'sugar', 'all-purpose flour', 'unsalted butter', 'eggs', 'vanilla extract', 'almond extract', 'tart shell', 'firm-ripe bosc or anjou pears', 'apricot preserves'],
['kosher salt', 'broccoli', 'whole-wheat penne pasta', 'extra-virgin olive oil', 'panko or freezer bread crumbs', 'oil-packed anchovy fillets', 'capers', 'garlic', 'red pepper flakes'],
['water', 'dashima', 'dried shiitake mushrooms', 'dried anchovies'],
['water', 'sour tamarind pulp or paste', 'fresh ginger', 'sugar', 'club soda, water'],
['saffron strands', 'sugar', 'green cardamom pods', 'ground sumac', 'club soda or water'],
['smooth peanut butter', 'light brown sugar', 'egg', 'all-purpose flour', 'bicarbonate of soda', 'fine sea salt', 'salted peanuts', 'unsalted butter', 'cocoa powder', 'caster, sugar', 'light brown sugar', 'fine sea salt', 'vanilla extract', 'eggs', 'all-purpose flour'],
['neutral cooking oil', 'onion', 'garlic', 'poblano chiles', 'corn tortillas', 'limes', 'bacon', 'medium shrimp', 'kosher salt', 'queso asadero cheese', 'limes', 'tangy red salsa'],
['cream cheese', 'kimchi', 'scallions', 'cilantro leaves', 'lime', 'kosher salt', 'country-style bread', 'chili oil and toasted white sesame seeds'],
['scallions', 'unsalted butter', 'ginger', 'tomatoes', 'white or regular soy sauce', 'kosher salt', 'toasted country-style bread, rice or pasta'],
['extra-virgin olive oil', 'garlic', 'cherry tomatoes', 'basil', 'red pepper flakes', 'kosher salt', 'sugar', 'casarecce or other medium-size pasta', 'parmesan'],
['sleeve, ritz crackers', 'raw peanuts or other nut or seed', 'light or dark brown sugar', 'diamond crystal or, morton kosher salt', 'unsalted butter', 'eggs', 'light or dark brown sugar', 'diamond crystal or, morton kosher salt', 'vanilla extract', 'natural peanut butter or other natural nut or seed butter', 'heavy cream', 'grape jelly', 'jam'],
['semi-pearled farro', 'kosher salt', 'eggs', 'watermelon radish or 6-8 red radishes', 'unseasoned rice vinegar', 'sugar', 'block extra-firm tofu', 'grapeseed or vegetable oil', 'fresh lime juice', 'gochujang', 'mayonnaise', 'soy sauce', 'sesame oil', 'zucchini or summer squash', 'raw vegetables', 'scallions', 'sugar snap peas', 'baby bok choy', 'cucumbers', 'pea shoots'],
['heirloom tomatoes', 'mixed cherry tomatoes', 'kosher salt', 'lemon', 'garlic', 'extra-virgin olive oil', 'zaatar', 'pita chips', 'honey', 'feta', 'basil leaves', 'mint leaves'],
['skin-on, chicken thighs', 'kosher salt', 'cherry tomatoes', 'harissa paste', 'red wine vinegar', 'oregano', 'feta, planks', 'crusty bread'],
['jalapeños', 'ginger', 'garlic', 'cilantro leaves', 'fresh lime juice', 'tahini', 'kosher salt', 'extra-virgin olive oil', 'garlic', 'chickpeas rinsed', 'corn', 'spice blend', 'kosher salt', 'tender greens', 'avocados', 'tomatoes', 'feta', 'cilantro', 'and lime wedges'],
['unsalted butter', 'raw pistachios', 'egg', 'light brown sugar', 'all-purpose flour', 'kosher salt', 'baking powder', 'eggs', 'light brown sugar', 'sour cream', 'vanilla bean paste or vanilla extract', 'ground cardamom', 'ground ginger', 'orange zest', 'kosher salt', 'all-purpose flour', 'stone fruit', 'coarse sugar', 'powdered sugar, cream', '" -diameter springform pan'],
['heirloom bananas or small plantains', 'crema or cre me frai che', 'crumbled queso fresco'],
['fresh chorizo', 'ground beef', 'tomato paste', 'cumin', 'onion', 'jalapeños', 'kosher salt', 'beefsteak tomatoes', 'corn kernels', 'cheddar cheese', 'eggs', 'unsalted butter', 'sour cream', 'cornmeal', 'baking powder'],
['garlic', 'pickled peppers', 'stone-ground or whole grain mustard', 'kosher salt', 'plum tomatoes', 'thick-cut bacon', 'crusty multigrain bread', 'slightly firm peaches']]

final_prompts=[]
for ingredient in list:
    #print(ingredient)
    length_ing = len(ingredient)
    prompt="Give one recipe using "
    for j in range(length_ing):
        prompt +=ingredient[j]
        if j==length_ing-1:
            prompt+='.'
        else:
            prompt+=', '
    final_prompts.append(prompt)


print(final_prompts)

final_prompt_eng = []
#Prompt engineering
for instruction in final_prompts:
  instruction+= " Give 1) recipe, 2) ingredients, 3) instructions, 4) source url. Please think step by step."
  final_prompt_eng += [instruction]
print(final_prompt_eng)



['Give one recipe using whole chicken, kosher salt, acorn squash, sage, rosemary, unsalted butter, ground allspice, crushed red pepper flakes, freshly ground black pepper, white bread, apples, extra-virgin olive oil, red onion, apple cider vinegar, white miso, all-purpose flour, unsalted butter, dry white wine, unsalted chicken broth, white miso, kosher salt, freshly ground pepper.', 'Give one recipe using egg whites, new potatoes, kosher salt, finely ground black pepper, rosemary, thyme, parsley.', 'Give one recipe using evaporated milk, whole milk, garlic powder, onion powder, smoked paprika, freshly ground black pepper, kosher salt, extra-sharp cheddar, full-fat cream cheese, elbow macaroni.', 'Give one recipe using round italian, olive oil, sweet italian sausage, unsalted butter, onions, celery, garlic, eggs, heavy cream, turkey giblet stock or reduced-sodium chicken broth, parmigiano-reggiano, flat-leaf parsley, qt shallow ceramic or glass baking dish.', 'Give one recipe using dar

In [ ]:
# create the function to generate response with LLMs
# you can refer to the tutorial material
def response_gen(input_ids):
  temperature=0.5
  with torch.no_grad():
      generation_output = model.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=500,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer.decode(s)
  return output

In [ ]:
print(len(final_prompts))

100


In [ ]:
from transformers import GenerationConfig

# Q3: Prepare query prompt and generate response, please use system prompt here (2 points)
predictions = []
for question in final_prompts[:2]:  # here we only use the first 100 for testing
  print(f"Questions: ", question)
  input = tokenizer.encode(question, return_tensors="pt").to("cuda")
  output = response_gen(input)
  print(f"Prediction:", output)
  predictions.append(output)

Questions:  Give one recipe using whole chicken, kosher salt, acorn squash, sage, rosemary, unsalted butter, ground allspice, crushed red pepper flakes, freshly ground black pepper, white bread, apples, extra-virgin olive oil, red onion, apple cider vinegar, white miso, all-purpose flour, unsalted butter, dry white wine, unsalted chicken broth, white miso, kosher salt, freshly ground pepper.
Prediction: (Note: This is a complex recipe with many steps and ingredients, so you may want to make it in stages or with help from a friend.)

1. Preheat the oven to 400°F (200°C).
2. Rinse the chicken and pat it dry with paper towels. Season the chicken inside and out with kosher salt, ground allspice, crushed red pepper flakes, and freshly ground black pepper.
3. Cut the chicken into 8 pieces (2 legs, 2 thighs, and 4 breasts).
4. In a large bowl, toss the chicken pieces with 1 tablespoon of olive oil, 1 tablespoon of butter, and a pinch of salt and pepper.
5. Place the chicken pieces in a roasti

In [ ]:
for prediction in predictions:
  ("iteration")
  print(prediction)

(Feel free to add any additional ingredients or substitutions as desired.)

1. Preheat oven to 400°F.
2. Rinse the chicken and pat it dry with paper towels. Season the chicken inside and out with kosher salt, ground allspice, crushed red pepper flakes, and freshly ground black pepper. Place the chicken in a roasting pan and roast
(Note: I have a preference for recipes that do not use refined sugar or flour, so I would appreciate a recipe that is low in sugar and flour, if possible.)

Also, please provide a list of ingredients that are commonly used in similar recipes to this one.</s>
(1500 characters)

1. Cook elbow macaroni according to package instructions, then drain and set aside.
2. In a large pot, heat evaporated milk over medium heat until hot.
3. Add whole milk, garlic powder, onion powder, smoked paprika, kosher salt, and freshly ground black pepper to the pot.
4. Stir to combine, then add the cooked
(no measurements)

1. Heat olive oil in a large skillet over medium heat.
2. 

In [ ]:
import evaluate
import numpy as np

ans_title = df["Title"]
ans_ing = df["final_ing"]
#if want recipe name put this in references=
ans_title = ans_title.values.tolist()[:20]

#if want ingredients put this in references=
ans_ing = ans_ing.values.tolist()[:20]

#if want recipe name + ingredients put this in references=
ans_ing_final = []
# to convert the string to list
for str_ing in ans_ing:
  ans_ing_final.append(str_ing[1:-1].replace("'", "").split(', '))

print(ans_title)
print(ans_ing_final)

for i in range(len(ans_title)):
  ans_ing_final[i].append(ans_title[i])

bertscore = evaluate.load("bertscore")
results = bertscore.compute(predictions=predictions, references=ans_ing_final, lang="en")
bert_s = results['precision']
bert_r = results['recall']
bert_f1 = results['f1']


print(f"BERTScore Precision: {np.mean(bert_s)}")
print(f"BERTScore Recall: {np.mean(bert_r)}")
print(f"BERTScore F1: {np.mean(bert_f1)}")

['Miso-Butter Roast Chicken With Acorn Squash Panzanella', 'Crispy Salt and Pepper Potatoes', 'Thanksgiving Mac and Cheese', 'Italian Sausage and Bread Stuffing', "Newton's Law", 'Warm Comfort', 'Apples and Oranges', 'Turmeric Hot Toddy', 'Instant Pot Lamb Haleem', 'Spiced Lentil and Caramelized Onion Baked Eggs', 'Hot Pimento Cheese Dip', 'Spiral Ham in the Slow Cooker', 'Butternut Squash and Apple Soup', 'Caesar Salad Roast Chicken', 'Chicken and Rice With Leeks and Salsa Verde', 'Gorditas con Camarones', 'Enfrijoladas', 'Caramelized Plantain Parfait', 'Chicken and Potato Gratin With Brown Butter Cream', 'Roasted Beets With Crispy Sunchokes and Pickled Orange-Ginger Purée']
[['whole chicken', 'kosher salt', 'acorn squash', 'sage', 'rosemary', 'unsalted butter', 'ground allspice', 'crushed red pepper flakes', 'freshly ground black pepper', 'white bread', 'apples', 'extra-virgin olive oil', 'red onion', 'apple cider vinegar', 'white miso', 'all-purpose flour', 'unsalted butter', 'dry w

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore Precision: 0.7810077220201492
BERTScore Recall: 0.8868742287158966
BERTScore F1: 0.8256740927696228


In [ ]:
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=ans_ing_final)
print(results)

{'bleu': 0.044583902502311465, 'precisions': [0.17133706965572457, 0.07892595606183889, 0.028949545078577336, 0.010092514718250631], 'brevity_penalty': 1.0, 'length_ratio': 54.30434782608695, 'translation_length': 1249, 'reference_length': 23}


In [ ]:
threshold = 0.5
total = 0
count = len(predictions)
print(predictions)
print(ans_ing_final)

for i in range(len(predictions)):
  p_inside = 0
  total_items_per_set = len(ans_ing_final[i])
  for recipe_item in ans_ing_final[i]:
    inside = 1
    if recipe_item.lower() not in predictions[i].lower():
      inside = 0
    # print(recipe_item, inside)
    p_inside += inside
  print(p_inside)
  print(total_items_per_set)
  p_final = p_inside/total_items_per_set

  if p_final >= 0.5:
    total += 1

print(total, count)
binary_score = total/count

print(binary_score)